In [1]:
import pandas as pd
from collections import defaultdict
import sys

In [2]:
finalfile="/Users/irffanalahi/Research/Research_code/gitignorefolder/MHB_per_read/moretest/readbaseddecon3/data/RealRun/collectmasterdf/tryingtotroubleshoot/comparecpgcoverage/sm100/mincpg3/nocov/BULK-PBMC-1389.bg_howsm_single_mode_pp_masterdf.txt_rej-99999.0_mincpg3_rejmode_nov_final.txt"
sm="/Users/irffanalahi/Research/Research_code/gitignorefolder/MHB_per_read/moretest/readbaseddecon3/data/RealRun/sm/blwithneu_hypo_stat.txt_0.01_2_g100.txtWithSurrounding.txt_1cpgnc.txt"
#celllist=["'CD14'","'CD19'","'CD4'","'CD8'","'CD56'","'Neu'","'EPCAM'"] 

finaldf=pd.read_csv(finalfile,sep="\t")
finaldf['acceptedCpG'] = finaldf.acceptedCpG.apply(lambda x: x[1:-1].split(','))
finaldf['CheckedCpG'] = finaldf.CheckedCpG.apply(lambda x: x[1:-1].split(','))

smdf=pd.read_csv(sm,sep="\t")
smdf['celltype']=smdf.celltype.apply(lambda x: x[1:-1].split(','))
finaldf.head()

,ReadName,allcelltype,ucelltype,assignedcelltype,CheckedCpG,mismatchCpG,notacceptedCpG,acceptedCpG,finalacceptedfor,finalrejectedfor,MateCIGARok
0,A00580:76:HFLG2DSXX:2:2252:8015:27680,[],[''],lowmapq,[],[''],[''],[''],notdetermined,notdetermined,Y
1,A00118:116:HJ7VFDSXX:3:1502:16947:25895,[],[''],lowmapq,[],[''],[''],[''],notdetermined,notdetermined,Y
2,A00580:76:HFLG2DSXX:4:1137:7853:36104,[],[''],lowmapq,[],[''],[''],[''],notdetermined,notdetermined,Y
3,A00580:76:HFLG2DSXX:2:1623:3920:33458,[],[''],NotAssigned,[],[''],[''],[''],notdetermined,notdetermined,Y
4,A00118:116:HJ7VFDSXX:3:2539:32226:16063,[],[''],lowmapq,[],[''],[''],[''],notdetermined,notdetermined,Y


In [3]:
smdf.head()

,chrom,start,end,celltype,blockchr,blockstart,blockend
0,chr15,52052102,52052104,['CD14'],chr15,52051953,52052253
1,chr3,122060546,122060548,['CD14'],chr3,122060397,122060721
2,chr16,79097508,79097510,['CD14'],chr16,79097359,79097659
3,chr1,159431113,159431115,['CD14'],chr1,159430964,159431264
4,chr12,48336372,48336374,['CD14'],chr12,48336223,48336523


In [4]:
celllist=(smdf['celltype']).tolist()
celllist=[x for sublist in celllist for x in sublist]
celllist=list(set(celllist))
print(celllist)

["'CD8'", "'CD19'", "'Neu'", "'CD56'", "'CD14'", "'CD4'"]


In [5]:
def countacccpg(sdf,df,cells):
    out={}
    for cell in cells:
        celldf=df[df['finalacceptedfor']==cell]
        
        correscpg=(celldf['acceptedCpG']).tolist()
        
        correscpg = [x for sublist in correscpg for x in sublist]
        
        correscpg = [i for i in correscpg if i] 
        
        correscpg=list(map(eval,correscpg))
        
        correscpgunique=list(set(correscpg))
        uniquelen=len(correscpgunique)
        
        #print(cell)
        #print(correscpgunique)
       
        out[cell]=uniquelen
    return out
    

In [6]:
def countcheckedCpG(sdf,df):
    allcheckedCpG=df['CheckedCpG']
    allcheckedCpG = [x for sublist in allcheckedCpG for x in sublist]
    allcheckedCpG=[i for i in allcheckedCpG if i] 
    allcheckedCpG=list(map(eval,allcheckedCpG))
   
    allcheckedCpG=list(set(allcheckedCpG))
   


    outdict=defaultdict(list)

    for cpg in allcheckedCpG:
        chrom,start=cpg.split(":")
        
        
        fromsm=sdf[(sdf['chrom']==chrom) & (sdf['start']==int(start))]
        if fromsm.shape[0]!=1:
            print("error. Exiting")
            sys.exit(1)
        
        celltype=fromsm['celltype']
        celltype=[x for sublist in celltype for x in sublist]
        if len(celltype)!=1:
            print("wrong.Exiting")
            sys.exit(1)
        outdict[celltype[0]].append(cpg)
        
    
    outdict=dict(outdict)
    outdictlen={}
    
    for k in outdict:
        outdictlen[k]=len(outdict[k])
    
    return outdictlen
    
    
        

In [7]:
acccpg=countacccpg(smdf,finaldf,celllist)

with open(finalfile+"_acccpgCount.txt", 'w') as f:
    print(acccpg, file=f)

print(acccpg)

{"'CD8'": 65, "'CD19'": 102, "'Neu'": 42, "'CD56'": 171, "'CD14'": 140, "'CD4'": 81}


In [8]:
checkedcpg=countcheckedCpG(smdf,finaldf)

with open(finalfile+"_checkedcpgCount.txt", 'w') as f:
    print(checkedcpg, file=f)

print(checkedcpg)

{"'Neu'": 297, "'CD19'": 390, "'CD4'": 302, "'CD56'": 373, "'CD8'": 282, "'CD14'": 299}
